In [ ]:
# !pip install import-ipynb

In [18]:
import pandas as pd
import import_ipynb
import preprocessamentofuncoes as pp

In [19]:
covidData = pd.read_csv("../../database/opendatasus/sp-px-opendata-2022.csv", on_bad_lines='skip')

/home/wallece/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15,20,21,22,26,29,30,31,33,34,35,36,41,45,46,48,50,51,53,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
# Visualizando Dados
covidData.head()
covidData.shape

(1000000, 65)

In [21]:
# Separando as colunas mais importantes
# colunas = covidData.columns

colunasImportantes = ['source_id', 'sexo', 'racaCor', 'idade', 'sintomas', 'condicoes',
                      'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal', 'evolucaoCaso']
covidData = covidData[colunasImportantes]

### Limpeza de Dados - Classificação Final


In [22]:
# Visualizando o formato das condições
covidData['classificacaoFinal'].value_counts()

Confirmado Laboratorial              152050
Descartado                           113197
Síndrome Gripal Não Especificada      50882
Confirmado Clínico-Epidemiológico      3249
Confirmado por Critério Clínico        3111
1                                       210
Confirmado Clínico-Imagem                96
2                                        26
Em tratamento domiciliar                  1
3                                         1
Cura                                      1
Name: classificacaoFinal, dtype: int64

In [23]:
covidData = pp.classFinPreprocessamento(covidData, 'classificacaoFinal')

In [24]:
covidData['classificacaoFinal'].value_counts()

Confirmado    158506
Name: classificacaoFinal, dtype: int64

In [25]:
dados_faltosos = pp.visualizarQntFaltosos(covidData)
dados_faltosos

source_id              0.000000
sexo                   0.000000
racaCor                0.022712
idade                  0.000000
sintomas               0.002524
condicoes             90.633793
dataPrimeiraDose      42.125219
dataSegundaDose       54.176498
classificacaoFinal     0.000000
evolucaoCaso           2.377197
dtype: float64

### Limpeza de Dados - Atibuto Evolução

In [26]:
# covidData[alvo].unique()
evolucao = 'evolucaoCaso'
covidData[evolucao].value_counts()

Cura                        128192
Em tratamento domiciliar     16838
Ignorado                      8233
Cancelado                     1067
Óbito                          264
Internado                      132
Internado em UTI                12
Name: evolucaoCaso, dtype: int64

In [29]:
covidData = pp.evolucaoPreprocessamento(covidData, evolucao)
covidData[evolucao].value_counts()

Cura         145030
Óbito           264
Internado       144
Name: evolucaoCaso, dtype: int64

In [30]:
dados_faltosos = pp.visualizarQntFaltosos(covidData)
dados_faltosos

source_id              0.000000
sexo                   0.000000
racaCor                0.023378
idade                  0.000000
sintomas               0.002750
condicoes             90.765825
dataPrimeiraDose      41.980775
dataSegundaDose       53.997580
classificacaoFinal     0.000000
evolucaoCaso           0.000000
dtype: float64

In [31]:
covidData[evolucao].value_counts()
# covidData.shape

Cura         145030
Óbito           264
Internado       144
Name: evolucaoCaso, dtype: int64

# Coletando os Sintomas

``` Pode conter - Assintomático, Febre, Dor de Garganta, Dispneia, Tosse,  Coriza, Dor de Cabeça, Distúrbios gustatórios, Distúrbios olfativos ```

In [32]:
# covidData[alvo].unique()
sintomas = 'sintomas'
covidData = pp.sintomasPreprocessamento(covidData, sintomas)
covidData.shape

TypeError: argument of type 'float' is not iterable

In [33]:
covidData.head()
# 'Febre' in covidData['sintomas'] 

,source_id,sexo,racaCor,idade,sintomas,condicoes,dataPrimeiraDose,dataSegundaDose,classificacaoFinal,evolucaoCaso
3,MAA0obKVBU,Feminino,Parda,20.0,"Dor de Cabeça, Outros",NaN,NaN,NaN,Confirmado,Cura
5,tZWluNQQl9,Masculino,Preta,43.0,"Febre, Tosse, Outros, Dispneia",NaN,NaN,NaN,Confirmado,Cura
9,LyB61WERfv,Feminino,Branca,60.0,"Febre, Tosse, Dispneia, Outros",NaN,NaN,NaN,Confirmado,Cura
14,8diqMhDO6j,Feminino,Branca,47.0,"Febre, Dor de Garganta, Outros, Tosse",NaN,2021-06-23,2021-08-02,Confirmado,Cura
25,lYfupZf9aJ,Feminino,Ignorado,54.0,"Tosse, Dor de Cabeça, Dispneia, Outros, Febre",NaN,NaN,NaN,Confirmado,Cura


# Organizando as Comorbidades
#### 

In [ ]:
# Comorbidades
comorbidades = {"diabetes": 'Diabetes', 
                "obesidade": 'Obesidade', 
                "renal": 'Doenças renais crônicas em estágio avançado',
                "respiratoria": 'Doenças respiratórias crônicas descompensadas',
                "imunossupressao": 'Imunossupressão',
                "fragilidadeImuno": 'Portador de doenças cromossômicas ou estado de fragilidade imunológica',
                "gestante": 'Gestante',
                "cardiaca": 'Doenças cardíacas crônicas',
                "puerpera": 'Puérpera'}

x = comorbidades.keys()


# stringX = 'Doenças cardíacas crônicas, Diabetes'
# comorbidades['diabetes'] in stringX

In [ ]:
for chave in comorbidades.keys():
    print(chave)

In [ ]:
# Substituindo o valor das comorbidades
covidNewData.loc[covidNewData.FATOR_RISC == 2, ["PUERPERA", "CARDIOPATI", "HEMATOLOGI",
            "SIND_DOWN", "HEPATICA", "ASMA","DIABETES", "NEUROLOGIC", "PNEUMOPATI",
            "IMUNODEPRE", "RENAL", "OBESIDADE"]] = 2;
covidNewData.RENAL.value_counts()

In [ ]:
covidNewData.shape

In [ ]:
newCovidData = covidNewData.drop(covidNewData[covidNewData.CLASSI_FIN != 5].index, inplace = False)
#covidNewData.shape
newCovidData.shape

In [ ]:
#Filtrando os atributos
attributes = ["CS_SEXO", "NU_IDADE_N", "CS_RACA", "SURTO_SG", "FEBRE", "TOSSE", "GARGANTA", 
              "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA","VOMITO", "FATOR_RISC", "PUERPERA", "CARDIOPATI", 
              "HEMATOLOGI", "SIND_DOWN", "HEPATICA", "ASMA", "DIABETES", "NEUROLOGIC","PNEUMOPATI",
              "IMUNODEPRE", "RENAL", "OBESIDADE", "DT_INTERNA", "DT_EVOLUCA", "CLASSI_FIN", "SUPORT_VEN", "UTI"]
# 'EVOLUCAO'

dbCovid = newCovidData[attributes];
dbCovid.shape

In [ ]:
write = pd.ExcelWriter('uti_dados_pp_Jan2021.xlsx')
dbCovid.to_excel(write, 'dados', index = False)
write.save()

In [ ]:
# Lista de Comorbidades
"""
Diabetes
Obesidade
Doenças renais crônicas em estágio avançado (graus 3, 4 e 5)
Doenças respiratórias crônicas descompensadas 
Imunossupressão
Portador de doenças cromossômicas ou estado de fragilidade imunológica
Gestante
Doenças cardíacas crônicas 
Puérpera (até 45 dias do parto)
"""